In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import interpolate, optimize
import ddm
import os 
import csv
import tensorflow as tf
import tensorflow_probability.python as tfp
from helper import Helper
from model_definitions import ModelTtaBounds, LossWLS

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])

In [2]:
def differential_evolution_tf(fitness, x_0, constraints):   
    def tensor_fitness(x_population):
        result = [fitness(x) for x in x_population]
        return tf.constant(result)

    tfp_result = tfp.optimizer.differential_evolution_minimize(tensor_fitness, 
                                                              initial_position=tf.constant(x_0),
                                                              max_iterations=400)    
    
    return optimize.OptimizeResult(x=[p.numpy() for p in tfp_result.position],
                             fun=tfp_result.objective_value.numpy(), 
                             nit=tfp_result.num_iterations.numpy(),
                             success=tfp_result.converged.numpy())

In [ ]:
model = ModelTtaBounds()
exp_data = pd.read_csv('../data/measures.csv', usecols=['subj_id', 'RT', 'is_turn_decision', 
                                                        'tta_condition', 'd_condition'])
subjects = exp_data.subj_id.unique()
conditions = [{'tta': tta, 'd': d} 
                  for tta in exp_data.tta_condition.unique() 
                  for d in exp_data.d_condition.unique()]

directory = '../model_fit_results/cross-validation'
Helper.write_to_csv(directory, 'model_%s_params_gpu.csv' % (model.model.name[0]), 
             ['subj_id', 'tta', 'd', 'loss'] + model.param_names)

subj_id = subjects[0]
condition = conditions[0]
fitted_model = Helper.fit_model(model.model, exp_data, subj_id, condition, lossfunction=LossWLS, 
                                      fitting_method=differential_evolution_tf)
Helper.write_to_csv(directory, 'model_%s_params_gpu.csv' % (model.model.name[0]), 
             [subj_id, condition['tta'], condition['d'], fit_model.get_fit_result().value()] 
             + fit_model.get_model_parameters())